In [1]:
from mytapering import *

In [2]:
import pennylane as qml
from pennylane import qchem

import numpy as np
from time import time

In [3]:
np.set_printoptions(precision=6)

# 1. Example: H2

In [4]:
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

In [5]:
# 基于JW变换构建的H2哈密顿量
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is \n", H)
print("-" * 20)
print("Number of terms: ", len(H.ops))

Number of qubits =  4
The Hamiltonian is 
   (-0.24274501260941383) [Z2]
+ (-0.24274501260941383) [Z3]
+ (-0.042072551947440084) [I0]
+ (0.1777135822909176) [Z0]
+ (0.1777135822909176) [Z1]
+ (0.12293330449299354) [Z0 Z2]
+ (0.12293330449299354) [Z1 Z3]
+ (0.1676833885560135) [Z0 Z3]
+ (0.1676833885560135) [Z1 Z2]
+ (0.17059759276836806) [Z0 Z1]
+ (0.17627661394181787) [Z2 Z3]
+ (-0.04475008406301996) [Y0 Y1 X2 X3]
+ (-0.04475008406301996) [X0 X1 Y2 Y3]
+ (0.04475008406301996) [Y0 X1 X2 Y3]
+ (0.04475008406301996) [X0 Y1 Y2 X3]
--------------------
Number of terms:  15


## 1.1 Tapering using QML

In [6]:
begin = time()
generators_by_qml = qml.symmetry_generators(H)
paulixops_by_qml = qml.paulix_ops(generators_by_qml, qubits)

n_electrons = 2
paulix_sector_by_qml = qml.qchem.optimal_sector(H, generators_by_qml, n_electrons)

H_tapered = qml.taper(H, generators_by_qml, paulixops_by_qml, paulix_sector_by_qml)
print(f'time cost: {time() - begin}')

print("Number of qubit after tapering =", len(H_tapered.wires))
print("The tapered Hamiltonian is \n", H_tapered)

C:\Users\Lee\anaconda3\envs\qchem\lib\site-packages\pennylane\operation.py:1886: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
C:\Users\Lee\anaconda3\envs\qchem\lib\site-packages\pennylane\operation.py:2073: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


time cost: 0.5321645736694336
Number of qubit after tapering = 1
The tapered Hamiltonian is 
   ((-0.27643173133526805+0j)) [I0]
+ ((0.17900033625207976+0j)) [X0]
+ ((0.8409171898006624+0j)) [Z0]


In [7]:
Hami_reduced_by_qml = create_pauliwords_from_hamilton(H_tapered, len(H_tapered.wires))

-0.276431731335 I
 0.840917189801 Z
 0.179000336252 X


## 1.2 Tapering using mytarpering

In [14]:
Hami_ori = create_pauliwords_from_hamilton(H, qubits)

-0.042072551947 IIII
 0.177713582291 ZIII
 0.177713582291 IZII
 0.170597592768 ZZII
 0.044750084063 YXXY
-0.044750084063 YYXX
-0.044750084063 XXYY
 0.044750084063 XYYX
-0.242745012609 IIZI
 0.122933304493 ZIZI
-0.242745012609 IIIZ
 0.167683388556 ZIIZ
 0.167683388556 IZZI
 0.122933304493 IZIZ
 0.176276613942 IIZZ


In [15]:
begin = time()
Hami_reduced_by_tapering = tapering(Hami_ori, n_electrons=2)
print(f'time cost: {time() - begin}')
print("Number of qubit after tapering =", Hami_reduced_by_tapering.num_qubits)
print("The tapered Hamiltonian is \n", Hami_reduced_by_tapering)

time cost: 0.026230335235595703
Number of qubit after tapering = 1
The tapered Hamiltonian is 
 -0.27643173+0.00000000j I
0.84091719+0.00000000j Z
0.17900034+0.00000000j X


## 1.3 compare

### 1.3.1 generators and paulixops

In [16]:
for idx, generator in enumerate(generators_by_qml):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops_by_qml[idx]}")

generator 1:   (1.0) [Z0 Z1], paulix_op: PauliX(wires=[1])
generator 2:   (1.0) [Z0 Z2], paulix_op: PauliX(wires=[2])
generator 3:   (1.0) [Z0 Z3], paulix_op: PauliX(wires=[3])


In [17]:
generators_by_tapering_ = get_generators_from_kernel(
    kernel_of_E(create_parity_check_matrix_E(*create_binary_matrix_G(Hami_ori)))
)
generators_by_tapering, paulix_ops_by_tapering = get_paulix_ops_from_generators(generators_by_tapering_)

for idx, generator in enumerate(generators_by_tapering.terms):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulix_ops_by_tapering.terms[idx]}")

generator 1: 1.00000000 ZZII, paulix_op: 1.00000000 IXII
generator 2: 1.00000000 ZIZI, paulix_op: 1.00000000 IIXI
generator 3: 1.00000000 ZIIZ, paulix_op: 1.00000000 IIIX


### 1.3.2 optimal sectors

In [18]:
print(f'sectors by qml: {paulix_sector_by_qml}')
print(f'sectors by tapering: {optimal_sector(Hami_ori, generators_by_tapering, n_electrons)}')

sectors by qml: [1, -1, -1]
sectors by tapering: [1, -1, -1]


### 1.3.3 final eigen values

In [19]:
print('Test: Hami_reduced_by_qml\n')
print("Sorted Eigenvalues: ")
eigvals_reduced_by_qml = np.linalg.eigvals(Hami_reduced_by_qml.matrix)
print(np.sort(eigvals_reduced_by_qml))
print('min eigenvalue: ', min(eigvals_reduced_by_qml))

Test: Hami_reduced_by_qml

Sorted Eigenvalues: 
[-1.136189+0.j  0.583326+0.j]
min eigenvalue:  (-1.1361891625218794+0j)


In [20]:
print('Test: Hami_reduced_by_tapering\n')
print("Sorted Eigenvalues: ")
eigvals_reduced_by_tapering = np.linalg.eigvals(Hami_reduced_by_tapering.matrix)
print(np.sort(eigvals_reduced_by_tapering))
print('min eigenvalue: ', min(eigvals_reduced_by_tapering))

Test: Hami_reduced_by_tapering

Sorted Eigenvalues: 
[-1.136189+0.j  0.583326+0.j]
min eigenvalue:  (-1.1361891625218794+0j)


# 2. Example: LiH

In [21]:
# https://github.com/aoterodelaroza/refdata/blob/master/20_g3/lih.xyz
symbols = ["Li", "H"]
coordinates = np.array([0.0, 0.0, 0.403635, 0.0, 0.0, -1.210905])

In [22]:
# 基于JW变换构建的H2哈密顿量
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is \n", H)
print("-" * 20)
print("Number of terms: ", len(H.ops))

Number of qubits =  12
The Hamiltonian is 
   (-3.844095589078443) [I0]
+ (-0.38874433443438816) [Z11]
+ (-0.38874433443438805) [Z10]
+ (-0.21337079370277998) [Z9]
+ (-0.21337079370277995) [Z8]
+ (-0.21337079370270737) [Z7]
+ (-0.21337079370270734) [Z6]
+ (-0.18269688137599668) [Z5]
+ (-0.18269688137599663) [Z4]
+ (-0.08811650933952661) [Z3]
+ (-0.0881165093395265) [Z2]
+ (1.0657693942968418) [Z0]
+ (1.0657693942968425) [Z1]
+ (-0.0040328077024712255) [Y1 Y3]
+ (-0.0040328077024712255) [X1 X3]
+ (0.001121895797431289) [Y3 Y5]
+ (0.001121895797431289) [X3 X5]
+ (0.008829856445608452) [Y2 Y4]
+ (0.008829856445608452) [X2 X4]
+ (0.04515772747839051) [Y0 Y2]
+ (0.04515772747839051) [X0 X2]
+ (0.05516443343995296) [Z4 Z10]
+ (0.05516443343995296) [Z5 Z11]
+ (0.05992853198294553) [Z4 Z6]
+ (0.05992853198294553) [Z5 Z7]
+ (0.059928531982973314) [Z4 Z8]
+ (0.059928531982973314) [Z5 Z9]
+ (0.06101038170335723) [Z2 Z4]
+ (0.06101038170335723) [Z3 Z5]
+ (0.06195795786087671) [Z4 Z11]
+ (0.0619579

## 2.1 Tapering using QML

In [23]:
begin = time()
generators_by_qml = qml.symmetry_generators(H)
paulixops_by_qml = qml.paulix_ops(generators_by_qml, qubits)

n_electrons = 4
paulix_sector_by_qml = qml.qchem.optimal_sector(H, generators_by_qml, n_electrons)

H_tapered = qml.taper(H, generators_by_qml, paulixops_by_qml, paulix_sector_by_qml)
print(f'time cost: {time() - begin}')

print("Number of qubit after tapering =", len(H_tapered.wires))
print("The tapered Hamiltonian is \n", H_tapered)

time cost: 165.20137000083923
Number of qubit after tapering = 8
The tapered Hamiltonian is 
   ((-3.68762286203592+0j)) [I0]
+ ((-0.42674158740555956+0j)) [Z8]
+ ((-0.4267415874054145+0j)) [Z6]
+ ((-0.18269688137599657+0j)) [Z4]
+ ((-0.18269688137599657+0j)) [Z5]
+ ((-0.08811650933952657+0j)) [Z3]
+ ((-0.08811650933952643+0j)) [Z2]
+ ((-0.009731378415249032+0j)) [X5]
+ ((-0.0036224668189408647+0j)) [X0]
+ ((0.003329640392654342+0j)) [X6]
+ ((0.0033296403926558835+0j)) [X8]
+ ((0.009033626870508097+0j)) [X4]
+ ((0.030963141132992696+0j)) [X1]
+ ((1.0657693942968405+0j)) [Z0]
+ ((1.0657693942968416+0j)) [Z1]
+ ((-0.030731095558011737+0j)) [Y2 Y3]
+ ((-0.012871066449284539+0j)) [X4 Z5]
+ ((-0.0067935244209237486+0j)) [Y4 Y5]
+ ((-0.005052528617892372+0j)) [Y0 Y1]
+ ((-0.004032807702471221+0j)) [Y1 Y3]
+ ((-0.004032807702471221+0j)) [X1 X3]
+ ((-0.0026554679625305938+0j)) [X1 X2]
+ ((-0.002587745598082857+0j)) [Y5 Y8]
+ ((-0.002587745598082857+0j)) [X5 X8]
+ ((-0.0025877455980816587+0j)) 

In [25]:
Hami_reduced_by_qml = create_pauliwords_from_hamilton(H_tapered, 9)

-3.687622862036 IIIIIIIII
 1.065769394297 ZIIIIIIII
 1.065769394297 IZIIIIIII
 0.408201270075 ZZIIIIIII
 0.027247318533 YZYIIIIII
 0.045157727478 YIYIIIIII
 0.027247318533 XZXIIIIII
 0.045157727478 XIXIIIIII
 0.024240011239 YZZZYIIII
 0.028997285065 YIZZYIIII
 0.024240011239 XZZZXIIII
 0.028997285065 XIZZXIIII
 0.025836613617 XZIZIZZIZ
 0.030963141133 XIIZIZZIZ
-0.025836613617 XZZZZZIII
-0.030963141133 XIZZZZIII
 0.027247318533 IYZYIIIII
 0.045157727478 ZYZYIIIII
 0.027247318533 IXZXIIIII
 0.045157727478 ZXZXIIIII
 0.010434033303 YXXYIIIII
-0.010434033303 YYXXIIIII
-0.010434033303 XXYYIIIII
 0.010434033303 XYYXIIIII
-0.003299302880 YXIXYIIII
-0.003299302880 YYIYYIIII
-0.003299302880 XXIXXIIII
-0.003299302880 XYIYXIIII
-0.006617582989 XXZXIZZIZ
-0.006617582989 XYZYIZZIZ
 0.006617582989 XXIXZZIII
 0.006617582989 XYIYZZIII
 0.024240011239 IYZZZYIII
 0.028997285065 ZYZZZYIII
 0.024240011239 IXZZZXIII
 0.028997285065 ZXZZZXIII
 0.003299302880 YXXZZYIII
-0.003299302880 YYXZZXIII
-0.003299302

## 2.2 Tapering using mytarpering

In [26]:
Hami_ori = create_pauliwords_from_hamilton(H, qubits)

-3.844095589078 IIIIIIIIIIII
 1.065769394297 ZIIIIIIIIIII
 1.065769394297 IZIIIIIIIIII
 0.408201270075 ZZIIIIIIIIII
 0.027247318533 YZYIIIIIIIII
 0.045157727478 YIYIIIIIIIII
 0.027247318533 XZXIIIIIIIII
 0.045157727478 XIXIIIIIIIII
 0.024240011239 YZZZYIIIIIII
 0.028997285065 YIZZYIIIIIII
 0.024240011239 XZZZXIIIIIII
 0.028997285065 XIZZXIIIIIII
-0.025836613617 YZZZZZZZZZYI
-0.030963141133 YIZZZZZZZZYI
-0.025836613617 XZZZZZZZZZXI
-0.030963141133 XIZZZZZZZZXI
 0.027247318533 IYZYIIIIIIII
 0.045157727478 ZYZYIIIIIIII
 0.027247318533 IXZXIIIIIIII
 0.045157727478 ZXZXIIIIIIII
 0.010434033303 YXXYIIIIIIII
-0.010434033303 YYXXIIIIIIII
-0.010434033303 XXYYIIIIIIII
 0.010434033303 XYYXIIIIIIII
-0.003299302880 YXIXYIIIIIII
-0.003299302880 YYIYYIIIIIII
-0.003299302880 XXIXXIIIIIII
-0.003299302880 XYIYXIIIIIII
 0.006617582989 YXIXZZZZZZYI
 0.006617582989 YYIYZZZZZZYI
 0.006617582989 XXIXZZZZZZXI
 0.006617582989 XYIYZZZZZZXI
 0.024240011239 IYZZZYIIIIII
 0.028997285065 ZYZZZYIIIIII
 0.02424001123

In [27]:
begin = time()
Hami_reduced_by_tapering = tapering(Hami_ori, n_electrons)
print(f'time cost: {time() - begin}')
print("Number of qubit after tapering =", Hami_reduced_by_tapering.num_qubits)
print("The tapered Hamiltonian is \n", Hami_reduced_by_tapering)

time cost: 1.7066140174865723
Number of qubit after tapering = 8
The tapered Hamiltonian is 
 -3.68762286+0.00000000j IIIIIIII
1.06576939+0.00000000j ZIIIIIII
1.06576939+0.00000000j IZIIIIII
0.40820127+0.00000000j ZZIIIIII
0.02724732+0.00000000j YZYIIIII
0.04515773+0.00000000j YIYIIIII
0.02724732+0.00000000j XZXIIIII
0.04515773+0.00000000j XIXIIIII
0.02424001+0.00000000j YZZZYIII
0.02899729+0.00000000j YIZZYIII
0.02424001+0.00000000j XZZZXIII
0.02899729+0.00000000j XIZZXIII
0.02583661+0.00000000j XZIZIZZZ
0.03096314+0.00000000j XIIZIZZZ
-0.02583661+0.00000000j XZZZZZII
-0.03096314+0.00000000j XIZZZZII
0.02724732+0.00000000j IYZYIIII
0.04515773+0.00000000j ZYZYIIII
0.02724732+0.00000000j IXZXIIII
0.04515773+0.00000000j ZXZXIIII
0.01043403+0.00000000j YXXYIIII
-0.01043403+0.00000000j YYXXIIII
-0.01043403+0.00000000j XXYYIIII
0.01043403+0.00000000j XYYXIIII
-0.00329930+0.00000000j YXIXYIII
-0.00329930+0.00000000j YYIYYIII
-0.00329930+0.00000000j XXIXXIII
-0.00329930+0.00000000j XYIYXIII
-

## 2.3 compare

### 2.3.1 generators and paulixops

In [28]:
for idx, generator in enumerate(generators_by_qml):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops_by_qml[idx]}")

generator 1:   (1.0) [Z6 Z7], paulix_op: PauliX(wires=[7])
generator 2:   (1.0) [Z8 Z9], paulix_op: PauliX(wires=[9])
generator 3:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10], paulix_op: PauliX(wires=[10])
generator 4:   (1.0) [Z1 Z3 Z5 Z6 Z8 Z11], paulix_op: PauliX(wires=[11])


In [29]:
generators_by_tapering_ = get_generators_from_kernel(
    kernel_of_E(create_parity_check_matrix_E(*create_binary_matrix_G(Hami_ori)))
)
generators_by_tapering, paulix_ops_by_tapering = get_paulix_ops_from_generators(generators_by_tapering_)

for idx, generator in enumerate(generators_by_tapering.terms):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulix_ops_by_tapering.terms[idx]}")

generator 1: 1.00000000 IIIIIIZZIIII, paulix_op: 1.00000000 IIIIIIIXIIII
generator 2: 1.00000000 IIIIIIIIZZII, paulix_op: 1.00000000 IIIIIIIIIXII
generator 3: 1.00000000 ZIZIZIZIZIZI, paulix_op: 1.00000000 IIIIIIIIIIXI
generator 4: 1.00000000 IZIZIZZIZIIZ, paulix_op: 1.00000000 IIIIIIIIIIIX


### 2.3.2 optimal sectors

In [30]:
print(f'sectors by qml: {paulix_sector_by_qml}')
print(f'sectors by tapering: {optimal_sector(Hami_ori, generators_by_tapering, n_electrons)}')

sectors by qml: [1, 1, 1, 1]
sectors by tapering: [1, 1, 1, 1]


### 2.3.3 final eigen values

In [34]:
print('Test: Hami_reduced_by_qml\n')
eigvals_reduced_by_qml = np.linalg.eigvals(Hami_reduced_by_qml.matrix)
print('min eigenvalue: ', min(eigvals_reduced_by_qml))

Test: Hami_reduced_by_qml

min eigenvalue:  (-7.686812902258399+2.958247109450349e-30j)


In [35]:
print('Test: Hami_reduced_by_tapering\n')
eigvals_reduced_by_tapering = np.linalg.eigvals(Hami_reduced_by_tapering.matrix)
print('min eigenvalue: ', min(eigvals_reduced_by_tapering))

Test: Hami_reduced_by_tapering

min eigenvalue:  (-7.686812902258386+0j)


# 3. Example: H2O

In [36]:
# https://github.com/aoterodelaroza/refdata/blob/master/20_g3/lih.xyz
symbols = ["O", "H", "H"]
coordinates = np.array([0.0, 0.0, 0.118882, 0.0, 0.756653, -0.475529, 0.0, -0.756653, -0.475529])

In [37]:
# 基于JW变换构建的H2哈密顿量
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is \n", H)
print("-" * 20)
print("Number of terms: ", len(H.ops))

Number of qubits =  14
The Hamiltonian is 
   (-41.7155299387761) [I0]
+ (0.5850622252195492) [Z10]
+ (0.5850622252195492) [Z11]
+ (0.61931339074801) [Z12]
+ (0.6193133907480104) [Z13]
+ (1.5317896296077471) [Z8]
+ (1.5317896296077482) [Z9]
+ (1.5557964873210017) [Z6]
+ (1.5557964873210017) [Z7]
+ (1.610860163302403) [Z4]
+ (1.610860163302403) [Z5]
+ (2.004283632934837) [Z2]
+ (2.0042836329348384) [Z3]
+ (12.684194370490733) [Z0]
+ (12.684194370490733) [Z1]
+ (-0.10769798653007104) [Y0 Y2]
+ (-0.10769798653007104) [X0 X2]
+ (-0.0017547076881392477) [Y1 Y3]
+ (-0.0017547076881392477) [X1 X3]
+ (0.11466433091423167) [Z10 Z12]
+ (0.11466433091423167) [Z11 Z13]
+ (0.1247842526916471) [Z6 Z10]
+ (0.1247842526916471) [Z7 Z11]
+ (0.12849290220939635) [Z10 Z11]
+ (0.1290238247351076) [Z6 Z11]
+ (0.1290238247351076) [Z7 Z10]
+ (0.13065731403782913) [Z8 Z10]
+ (0.13065731403782913) [Z9 Z11]
+ (0.1319073137762945) [Z4 Z10]
+ (0.1319073137762945) [Z5 Z11]
+ (0.13268317826520623) [Z2 Z10]
+ (0.1326

## 3.1 Tapering using QML

In [39]:
begin = time()
generators_by_qml = qml.symmetry_generators(H)
paulixops_by_qml = qml.paulix_ops(generators_by_qml, qubits)

n_electrons = 10
paulix_sector_by_qml = qml.qchem.optimal_sector(H, generators_by_qml, n_electrons)

H_tapered = qml.taper(H, generators_by_qml, paulixops_by_qml, paulix_sector_by_qml)
print(f'time cost: {time() - begin}')

print("Number of qubit after tapering =", len(H_tapered.wires))
print("The tapered Hamiltonian is \n", H_tapered)

time cost: 80.01912212371826
Number of qubit after tapering = 11
The tapered Hamiltonian is 
   ((-41.49549016637345+0j)) [I0]
+ ((0.005121627837741981+0j)) [X8]
+ ((0.5850622252195489+0j)) [Z10]
+ ((0.5850622252195489+0j)) [Z11]
+ ((1.5557964873210008+0j)) [Z6]
+ ((1.5557964873210008+0j)) [Z7]
+ ((1.610860163302402+0j)) [Z4]
+ ((1.610860163302402+0j)) [Z5]
+ ((2.004283632934836+0j)) [Z2]
+ ((2.0042836329348375+0j)) [Z3]
+ ((3.0635792592154942+0j)) [Z8]
+ ((12.684194370490728+0j)) [Z0]
+ ((12.684194370490728+0j)) [Z1]
+ ((-0.107697986530071+0j)) [Y0 Y2]
+ ((-0.107697986530071+0j)) [X0 X2]
+ ((-0.03356262278066429+0j)) [Y4 Y5]
+ ((-0.02182623221731727+0j)) [Y10 Y11]
+ ((-0.017023869115153453+0j)) [Y6 Y7]
+ ((-0.01147677266101199+0j)) [Y2 Y3]
+ ((-0.010604076936661527+0j)) [X7 X10]
+ ((-0.0075817932778138595+0j)) [Y0 Y1]
+ ((-0.0032894817809553953+0j)) [Y0 Y11]
+ ((-0.0017547076881392469+0j)) [Y1 Y3]
+ ((-0.0017547076881392469+0j)) [X1 X3]
+ ((0.0028818112373080245+0j)) [X1 X2]
+ ((0.003

In [41]:
Hami_reduced_by_qml = create_pauliwords_from_hamilton(H_tapered, 12)

-41.495490166373 IIIIIIIIIIII
 12.684194370491 ZIIIIIIIIIII
 12.684194370491 IZIIIIIIIIII
 1.184063956791 ZZIIIIIIIIII
-0.138405064313 YZYIIIIIIIII
-0.107697986530 YIYIIIIIIIII
-0.138405064313 XZXIIIIIIIII
-0.107697986530 XIXIIIIIIIII
 0.051575032412 YZZZZZYIIIII
 0.046386627497 YIZZZZYIIIII
 0.051575032412 XZZZZZXIIIII
 0.046386627497 XIZZZZXIIIII
 0.084661213761 YZZZZZZZIIYI
 0.071895206387 YIZZZZZZIIYI
 0.084661213761 XZZZZZZZIIXI
 0.071895206387 XIZZZZZZIIXI
-0.138405064313 IYZYIIIIIIII
-0.107697986530 ZYZYIIIIIIII
-0.138405064313 IXZXIIIIIIII
-0.107697986530 ZXZXIIIIIIII
 0.016435557940 YXXYIIIIIIII
-0.016435557940 YYXXIIIIIIII
-0.016435557940 XXYYIIIIIIII
 0.016435557940 XYYXIIIIIIII
 0.004355168006 YXIXZZYIIIII
 0.004355168006 YYIYZZYIIIII
 0.004355168006 XXIXZZXIIIII
 0.004355168006 XYIYZZXIIIII
 0.011854651773 YXIXZZZZIIYI
 0.011854651773 YYIYZZZZIIYI
 0.011854651773 XXIXZZZZIIXI
 0.011854651773 XYIYZZZZIIXI
 0.004240435418 YXIIXYIIIIII
-0.004240435418 YYIIXXIIIIII
-0.00424043

## 3.2 Tapering using mytarpering

In [42]:
Hami_ori = create_pauliwords_from_hamilton(H, qubits)

-41.715529938776 IIIIIIIIIIIIII
 12.684194370491 ZIIIIIIIIIIIII
 12.684194370491 IZIIIIIIIIIIII
 1.184063956791 ZZIIIIIIIIIIII
-0.138405064313 YZYIIIIIIIIIII
-0.107697986530 YIYIIIIIIIIIII
-0.138405064313 XZXIIIIIIIIIII
-0.107697986530 XIXIIIIIIIIIII
 0.051575032412 YZZZZZYIIIIIII
 0.046386627497 YIZZZZYIIIIIII
 0.051575032412 XZZZZZXIIIIIII
 0.046386627497 XIZZZZXIIIIIII
 0.084661213761 YZZZZZZZZZYIII
 0.071895206387 YIZZZZZZZZYIII
 0.084661213761 XZZZZZZZZZXIII
 0.071895206387 XIZZZZZZZZXIII
-0.138405064313 IYZYIIIIIIIIII
-0.107697986530 ZYZYIIIIIIIIII
-0.138405064313 IXZXIIIIIIIIII
-0.107697986530 ZXZXIIIIIIIIII
 0.016435557940 YXXYIIIIIIIIII
-0.016435557940 YYXXIIIIIIIIII
-0.016435557940 XXYYIIIIIIIIII
 0.016435557940 XYYXIIIIIIIIII
 0.004355168006 YXIXZZYIIIIIII
 0.004355168006 YYIYZZYIIIIIII
 0.004355168006 XXIXZZXIIIIIII
 0.004355168006 XYIYZZXIIIIIII
 0.011854651773 YXIXZZZZZZYIII
 0.011854651773 YYIYZZZZZZYIII
 0.011854651773 XXIXZZZZZZXIII
 0.011854651773 XYIYZZZZZZXIII
 0.00

In [43]:
begin = time()
Hami_reduced_by_tapering = tapering(Hami_ori, n_electrons)
print(f'time cost: {time() - begin}')
print("Number of qubit after tapering =", Hami_reduced_by_tapering.num_qubits)
print("The tapered Hamiltonian is \n", Hami_reduced_by_tapering)

time cost: 3.968219518661499
Number of qubit after tapering = 11
The tapered Hamiltonian is 
 -41.49549017+0.00000000j IIIIIIIIIII
12.68419437+0.00000000j ZIIIIIIIIII
12.68419437+0.00000000j IZIIIIIIIII
1.18406396+0.00000000j ZZIIIIIIIII
-0.13840506+0.00000000j YZYIIIIIIII
-0.10769799+0.00000000j YIYIIIIIIII
-0.13840506+0.00000000j XZXIIIIIIII
-0.10769799+0.00000000j XIXIIIIIIII
0.05157503+0.00000000j YZZZZZYIIII
0.04638663+0.00000000j YIZZZZYIIII
0.05157503+0.00000000j XZZZZZXIIII
0.04638663+0.00000000j XIZZZZXIIII
0.08466121+0.00000000j YZZZZZZZIYI
0.07189521+0.00000000j YIZZZZZZIYI
0.08466121+0.00000000j XZZZZZZZIXI
0.07189521+0.00000000j XIZZZZZZIXI
-0.13840506+0.00000000j IYZYIIIIIII
-0.10769799+0.00000000j ZYZYIIIIIII
-0.13840506+0.00000000j IXZXIIIIIII
-0.10769799+0.00000000j ZXZXIIIIIII
0.01643556+0.00000000j YXXYIIIIIII
-0.01643556+0.00000000j YYXXIIIIIII
-0.01643556+0.00000000j XXYYIIIIIII
0.01643556+0.00000000j XYYXIIIIIII
0.00435517+0.00000000j YXIXZZYIIII
0.00435517+0.0000

## 3.3 compare

### 3.3.1 generators and paulixops

In [46]:
for idx, generator in enumerate(generators_by_qml):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops_by_qml[idx]}")

generator 1:   (1.0) [Z8 Z9], paulix_op: PauliX(wires=[9])
generator 2:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10 Z12], paulix_op: PauliX(wires=[12])
generator 3:   (1.0) [Z1 Z3 Z5 Z7 Z8 Z11 Z13], paulix_op: PauliX(wires=[13])


In [47]:
generators_by_tapering_ = get_generators_from_kernel(
    kernel_of_E(create_parity_check_matrix_E(*create_binary_matrix_G(Hami_ori)))
)
generators_by_tapering, paulix_ops_by_tapering = get_paulix_ops_from_generators(generators_by_tapering_)

for idx, generator in enumerate(generators_by_tapering.terms):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulix_ops_by_tapering.terms[idx]}")

generator 1: 1.00000000 IIIIIIIIZZIIII, paulix_op: 1.00000000 IIIIIIIIIXIIII
generator 2: 1.00000000 ZIZIZIZIZIZIZI, paulix_op: 1.00000000 IIIIIIIIIIIIXI
generator 3: 1.00000000 IZIZIZIZZIIZIZ, paulix_op: 1.00000000 IIIIIIIIIIIIIX


### 3.3.2 optimal sectors

In [49]:
print(f'sectors by qml: {paulix_sector_by_qml}')
print(f'sectors by tapering: {optimal_sector(Hami_ori, generators_by_tapering, n_electrons)}')

sectors by qml: [1, -1, -1]
sectors by tapering: [1, -1, -1]


# 4. Example: BeH2

In [52]:
# http://ursula.chem.yale.edu/~chem220/chem220js/STUDYAIDS/hybridization/mo/beh2.log
symbols = ["Be", "H", "H"]
coordinates = np.array([-0.49317, 1.16085, 0.0, 0.69683, 1.16085, 0.0, -1.68317, 1.16085, 0.0])

In [53]:
# 基于JW变换构建的H2哈密顿量
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is \n", H)
print("-" * 20)
print("Number of terms: ", len(H.ops))

Number of qubits =  14
The Hamiltonian is 
   (-6.311794785724469) [I0]
+ (-0.8171638849443204) [Z13]
+ (-0.8171638849443201) [Z12]
+ (-0.3412642728703804) [Z10]
+ (-0.3412642728703803) [Z11]
+ (-0.07268403369472537) [Z7]
+ (-0.07268403369472531) [Z6]
+ (-0.07268403369472362) [Z9]
+ (-0.07268403369472358) [Z8]
+ (0.08400532522364187) [Z3]
+ (0.08400532522364221) [Z2]
+ (0.09592687719026514) [Z4]
+ (0.09592687719026514) [Z5]
+ (2.4673468849172937) [Z0]
+ (2.4673468849172946) [Z1]
+ (0.07717080613357652) [Z10 Z12]
+ (0.07717080613357652) [Z11 Z13]
+ (0.08687237248084279) [Z2 Z4]
+ (0.08687237248084279) [Z3 Z5]
+ (0.08998679138300451) [Z4 Z6]
+ (0.08998679138300451) [Z5 Z7]
+ (0.08998679138302458) [Z4 Z8]
+ (0.08998679138302458) [Z5 Z9]
+ (0.09124508190558271) [Z6 Z10]
+ (0.09124508190558271) [Z7 Z11]
+ (0.09124508190560301) [Z8 Z10]
+ (0.09124508190560301) [Z9 Z11]
+ (0.09150767112431096) [Z6 Z12]
+ (0.09150767112431096) [Z7 Z13]
+ (0.09150767112433156) [Z8 Z12]
+ (0.09150767112433156) [

## 4.1 Tapering using QML

In [55]:
begin = time()
generators_by_qml = qml.symmetry_generators(H)
paulixops_by_qml = qml.paulix_ops(generators_by_qml, qubits)

n_electrons = 6
paulix_sector_by_qml = qml.qchem.optimal_sector(H, generators_by_qml, n_electrons)

H_tapered = qml.taper(H, generators_by_qml, paulixops_by_qml, paulix_sector_by_qml)
print(f'time cost: {time() - begin}')

print("Number of qubit after tapering =", len(H_tapered.wires))
print("The tapered Hamiltonian is \n", H_tapered)

time cost: 892.9166102409363
Number of qubit after tapering = 9
The tapered Hamiltonian is 
   ((-6.086865240604816+0j)) [I0]
+ ((-0.34126427287038+0j)) [Z10]
+ ((-0.14536806738945063+0j)) [Z6]
+ ((-0.1453680673894471+0j)) [Z8]
+ ((-0.024996812058313084+0j)) [X5]
+ ((0.00239851193788527+0j)) [X6]
+ ((0.00239851193788583+0j)) [X8]
+ ((0.0840053252236418+0j)) [Z3]
+ ((0.08400532522364214+0j)) [Z2]
+ ((0.09592687719026513+0j)) [Z4]
+ ((0.09592687719026513+0j)) [Z5]
+ ((2.467346884917291+0j)) [Z0]
+ ((2.467346884917292+0j)) [Z1]
+ ((-0.0437109433902769+0j)) [Y2 Y3]
+ ((-0.02198098982547869+0j)) [Y4 Y10]
+ ((-0.02198098982547869+0j)) [X4 X10]
+ ((-0.015932894580459225+0j)) [X5 X10]
+ ((-0.015276353816802757+0j)) [X5 Z10]
+ ((-0.011747610873581945+0j)) [Y4 Y5]
+ ((-0.0008727725039831971+0j)) [Y0 Y1]
+ ((0.00018149288502513687-0j)) [Y1 Y3]
+ ((0.0020334594503994743+0j)) [Y1 Y4]
+ ((0.0028436504083918335-0j)) [Y1 Y2]
+ ((0.00789070935859554+0j)) [X6 X10]
+ ((0.00789070935859554+0j)) [Y6 Y10]
+

In [58]:
Hami_reduced_by_qml = create_pauliwords_from_hamilton(H_tapered, 11)

-6.086865240605 IIIIIIIIIII
 2.467346884917 ZIIIIIIIIII
 2.467346884917 IZIIIIIIIII
 0.559461224125 ZZIIIIIIIII
 0.067832693201 YZZZYIIIIII
 0.078415863800 YIZZYIIIIII
 0.067832693201 XZZZXIIIIII
 0.078415863800 XIZZXIIIIII
 0.048277063636 YZZZZZIIIIY
 0.043293922289 YIZZZZIIIIY
 0.048277063636 XZZZZZIIIIX
 0.043293922289 XIZZZZIIIIX
 0.004668976323 YXXYIIIIIII
-0.004668976323 YYXXIIIIIII
-0.004668976323 XXYYIIIIIII
 0.004668976323 XYYXIIIIIII
-0.000163938150 YYZXZIZIZIZ
 0.000163938150 YXZYZIZIZIZ
-0.000163938150 YYIXIIIIIII
 0.000163938150 YXIYIIIIIII
 0.067832693201 IYZZZYIIIII
 0.078415863800 ZYZZZYIIIII
 0.067832693201 IXZZZXIIIII
 0.078415863800 ZXZZZXIIIII
 0.018021322949 YXIIXYIIIII
-0.018021322949 YYIIXXIIIII
-0.018021322949 XXIIYYIIIII
 0.018021322949 XYIIYXIIIII
-0.010764826109 YXIIIXIIIIY
-0.010764826109 YYIIIYIIIIY
-0.010764826109 XXIIIXIIIIX
-0.010764826109 XYIIIYIIIIX
 0.003964016765 YXIIIIYIIII
-0.003964016765 YYIIIIXIIII
 0.003964016765 XXIIIIXIIII
 0.003964016765 XYII

## 4.2 Tapering using mytarpering

In [59]:
Hami_ori = create_pauliwords_from_hamilton(H, qubits)

-6.311794785724 IIIIIIIIIIIIII
 2.467346884917 ZIIIIIIIIIIIII
 2.467346884917 IZIIIIIIIIIIII
 0.559461224125 ZZIIIIIIIIIIII
 0.067832693201 YZZZYIIIIIIIII
 0.078415863800 YIZZYIIIIIIIII
 0.067832693201 XZZZXIIIIIIIII
 0.078415863800 XIZZXIIIIIIIII
 0.048277063636 YZZZZZZZZZYIII
 0.043293922289 YIZZZZZZZZYIII
 0.048277063636 XZZZZZZZZZXIII
 0.043293922289 XIZZZZZZZZXIII
 0.004668976323 YXXYIIIIIIIIII
-0.004668976323 YYXXIIIIIIIIII
-0.004668976323 XXYYIIIIIIIIII
 0.004668976323 XYYXIIIIIIIIII
-0.000163938150 YXIXZZZZZZZZYI
-0.000163938150 YYIYZZZZZZZZYI
-0.000163938150 XXIXZZZZZZZZXI
-0.000163938150 XYIYZZZZZZZZXI
 0.067832693201 IYZZZYIIIIIIII
 0.078415863800 ZYZZZYIIIIIIII
 0.067832693201 IXZZZXIIIIIIII
 0.078415863800 ZXZZZXIIIIIIII
 0.018021322949 YXIIXYIIIIIIII
-0.018021322949 YYIIXXIIIIIIII
-0.018021322949 XXIIYYIIIIIIII
 0.018021322949 XYIIYXIIIIIIII
-0.010764826109 YXIIIXZZZZYIII
-0.010764826109 YYIIIYZZZZYIII
-0.010764826109 XXIIIXZZZZXIII
-0.010764826109 XYIIIYZZZZXIII
 0.00396

In [60]:
begin = time()
Hami_reduced_by_tapering = tapering(Hami_ori, n_electrons)
print(f'time cost: {time() - begin}')
print("Number of qubit after tapering =", Hami_reduced_by_tapering.num_qubits)
print("The tapered Hamiltonian is \n", Hami_reduced_by_tapering)

time cost: 1.999345302581787
Number of qubit after tapering = 9
The tapered Hamiltonian is 
 -6.08686524+0.00000000j IIIIIIIII
2.46734688+0.00000000j ZIIIIIIII
2.46734688+0.00000000j IZIIIIIII
0.55946122+0.00000000j ZZIIIIIII
0.06783269+0.00000000j YZZZYIIII
0.07841586+0.00000000j YIZZYIIII
0.06783269+0.00000000j XZZZXIIII
0.07841586+0.00000000j XIZZXIIII
0.04827706+0.00000000j YZZZZZIIY
0.04329392+0.00000000j YIZZZZIIY
0.04827706+0.00000000j XZZZZZIIX
0.04329392+0.00000000j XIZZZZIIX
0.00466898+0.00000000j YXXYIIIII
-0.00466898+0.00000000j YYXXIIIII
-0.00466898+0.00000000j XXYYIIIII
0.00466898+0.00000000j XYYXIIIII
0.00016394+0.00000000j YYZXZIZZZ
-0.00016394+0.00000000j YXZYZIZZZ
0.00016394+0.00000000j YYIXIIIII
-0.00016394+0.00000000j YXIYIIIII
0.06783269+0.00000000j IYZZZYIII
0.07841586+0.00000000j ZYZZZYIII
0.06783269+0.00000000j IXZZZXIII
0.07841586+0.00000000j ZXZZZXIII
0.01802132+0.00000000j YXIIXYIII
-0.01802132+0.00000000j YYIIXXIII
-0.01802132+0.00000000j XXIIYYIII
0.0180213

## 4.3 compare

### 4.3.1 generators and paulixops

In [61]:
for idx, generator in enumerate(generators_by_qml):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops_by_qml[idx]}")

generator 1:   (1.0) [Z6 Z7], paulix_op: PauliX(wires=[7])
generator 2:   (1.0) [Z8 Z9], paulix_op: PauliX(wires=[9])
generator 3:   (1.0) [Z0 Z1 Z4 Z5 Z10 Z11], paulix_op: PauliX(wires=[11])
generator 4:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10 Z12], paulix_op: PauliX(wires=[12])
generator 5:   (1.0) [Z0 Z3 Z4 Z6 Z8 Z10 Z13], paulix_op: PauliX(wires=[13])


In [62]:
generators_by_tapering_ = get_generators_from_kernel(
    kernel_of_E(create_parity_check_matrix_E(*create_binary_matrix_G(Hami_ori)))
)
generators_by_tapering, paulix_ops_by_tapering = get_paulix_ops_from_generators(generators_by_tapering_)

for idx, generator in enumerate(generators_by_tapering.terms):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulix_ops_by_tapering.terms[idx]}")

generator 1: 1.00000000 IIIIIIZZIIIIII, paulix_op: 1.00000000 IIIIIIIXIIIIII
generator 2: 1.00000000 IIIIIIIIZZIIII, paulix_op: 1.00000000 IIIIIIIIIXIIII
generator 3: 1.00000000 ZZIIZZIIIIZZII, paulix_op: 1.00000000 IIIIIIIIIIIXII
generator 4: 1.00000000 ZIZIZIZIZIZIZI, paulix_op: 1.00000000 IIIIIIIIIIIIXI
generator 5: 1.00000000 ZIIZZIZIZIZIIZ, paulix_op: 1.00000000 IIIIIIIIIIIIIX


### 4.3.2 optimal sectors

In [63]:
print(f'sectors by qml: {paulix_sector_by_qml}')
print(f'sectors by tapering: {optimal_sector(Hami_ori, generators_by_tapering, n_electrons)}')

sectors by qml: [1, 1, 1, -1, -1]
sectors by tapering: [1, 1, 1, -1, -1]


# 5. Example: H6

In [4]:
symbols = ["H", "H", "H", "H", "H", "H"]
coordinates = np.array([
    0.0, 0.0, 0.0,
    0.0, 0.0, 0.5,
    0.0, 0.0, 1.0,
    0.0, 0.0, 1.5,
    0.0, 0.0, 2.0,
    0.0, 0.0, 2.5
])

In [5]:
# 基于JW变换构建的H2哈密顿量
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is \n", H)
print("-" * 20)
print("Number of terms: ", len(H.ops))

Number of qubits =  12
The Hamiltonian is 
   (-5.517857995732383) [Z10]
+ (-5.517857995732383) [Z11]
+ (-2.975845114540733) [Z8]
+ (-2.975845114540733) [Z9]
+ (-1.2016688151928356) [Z6]
+ (-1.2016688151928352) [Z7]
+ (-0.3251277147960169) [Z4]
+ (-0.3251277147960169) [Z5]
+ (0.1581048642489347) [Z3]
+ (0.1581048642489349) [Z2]
+ (0.45859219444170546) [Z1]
+ (0.45859219444170574) [Z0]
+ (23.712173132473378) [I0]
+ (-3.022426753318541e-11) [Y9 Y11]
+ (-3.022426753318541e-11) [X9 X11]
+ (-1.2464695942071558e-11) [Y8 Y10]
+ (-1.2464695942071558e-11) [X8 X10]
+ (-3.4363623058197845e-12) [Y6 Y8]
+ (-3.4363623058197845e-12) [X6 X8]
+ (-1.2825296380469808e-12) [Y7 Y9]
+ (-1.2825296380469808e-12) [X7 X9]
+ (0.11976424626602511) [Z2 Z4]
+ (0.11976424626602511) [Z3 Z5]
+ (0.1330796508182468) [Z0 Z2]
+ (0.1330796508182468) [Z1 Z3]
+ (0.13456238001270737) [Z4 Z6]
+ (0.13456238001270737) [Z5 Z7]
+ (0.13674367059913745) [Z0 Z4]
+ (0.13674367059913745) [Z1 Z5]
+ (0.14245208941941903) [Z2 Z6]
+ (0.142

## 5.1 Tapering using QML

In [14]:
begin = time()
generators_by_qml = qml.symmetry_generators(H)
paulixops_by_qml = qml.paulix_ops(generators_by_qml, qubits)

n_electrons = 6
paulix_sector_by_qml = qml.qchem.optimal_sector(H, generators_by_qml, n_electrons)

H_tapered = qml.taper(H, generators_by_qml, paulixops_by_qml, paulix_sector_by_qml)
print(f'time cost: {time() - begin}')

print("Number of qubit after tapering =", len(H_tapered.wires))
print("The tapered Hamiltonian is \n", H_tapered)

time cost: 17.71953511238098
Number of qubit after tapering = 10
The tapered Hamiltonian is 
   ((-2.975845114540732+0j)) [Z8]
+ ((-2.975845114540732+0j)) [Z9]
+ ((-1.2016688151928352+0j)) [Z6]
+ ((-1.2016688151928347+0j)) [Z7]
+ ((-0.32512771479601676+0j)) [Z4]
+ ((-0.32512771479601676+0j)) [Z5]
+ ((3.1719071813540706e-12-0j)) [X0]
+ ((1.2464695942071551e-11-0j)) [X8]
+ ((3.02242675331854e-11-0j)) [X9]
+ ((0.15810486424893463+0j)) [Z3]
+ ((0.15810486424893486+0j)) [Z2]
+ ((0.45859219444170524+0j)) [Z1]
+ ((0.4585921944417055+0j)) [Z0]
+ ((23.71217313247337+0j)) [I0]
+ ((-0.048109725771599415+0j)) [Y8 Y9]
+ ((-0.029896558031927242+0j)) [Y6 Y7]
+ ((-0.017853382465998317+0j)) [Y4 Y5]
+ ((-0.015017052424719027+0j)) [Y0 Y1]
+ ((-0.013500019685003018+0j)) [Y2 Y3]
+ ((-6.3199328582952375e-12+0j)) [Z0 X1]
+ ((-4.90540941200379e-12+0j)) [X7 X8]
+ ((-3.436362305819783e-12+0j)) [Y6 Y8]
+ ((-3.436362305819783e-12+0j)) [X6 X8]
+ ((-2.372078228285445e-12+0j)) [X1 X2]
+ ((5.8071377567747685e-12-0j))

## 5.2 Tapering using mytarpering

In [6]:
Hami_ori = create_pauliwords_from_hamilton(H, qubits)

 23.712173132473 IIIIIIIIIIII
 0.458592194442 ZIIIIIIIIIII
 0.458592194442 IZIIIIIIIIII
 0.197358018668 ZZIIIIIIIIII
 0.102960957779 YZZZYIIIIIII
-0.034392300628 YIZZYIIIIIII
 0.102960957779 XZZZXIIIIIII
-0.034392300628 XIZZXIIIIIII
 0.000000000007 YZZZZZYIIIII
 0.000000000007 XZZZZZXIIIII
-0.032439078404 YZZZZZZZYIII
 0.005968376363 YIZZZZZZYIII
-0.032439078404 XZZZZZZZXIII
 0.005968376363 XIZZZZZZXIII
-0.000000000006 YZZZZZZZZZYI
 0.000000000001 YIZZZZZZZZYI
-0.000000000006 XZZZZZZZZZXI
 0.000000000001 XIZZZZZZZZXI
 0.031020123804 YXXYIIIIIIII
-0.031020123804 YYXXIIIIIIII
-0.031020123804 XXYYIIIIIIII
 0.031020123804 XYYXIIIIIIII
-0.018781085028 YXIXZZYIIIII
-0.018781085028 YYIYZZYIIIII
-0.018781085028 XXIXZZXIIIII
-0.018781085028 XYIYZZXIIIII
 0.002856074535 YXIXZZZZZZYI
 0.002856074535 YYIYZZZZZZYI
 0.002856074535 XXIXZZZZZZXI
 0.002856074535 XYIYZZZZZZXI
 0.102960957779 IYZZZYIIIIII
-0.034392300628 ZYZZZYIIIIII
 0.102960957779 IXZZZXIIIIII
-0.034392300628 ZXZZZXIIIIII
 0.0210436150

In [7]:
begin = time()
Hami_reduced_by_tapering = tapering(Hami_ori, 6)
print(f'time cost: {time() - begin}')
print("Number of qubit after tapering =", Hami_reduced_by_tapering.num_qubits)
print("The tapered Hamiltonian is \n", Hami_reduced_by_tapering)

time cost: 2.0893712043762207
Number of qubit after tapering = 10
The tapered Hamiltonian is 
 23.71217313+0.00000000j IIIIIIIIII
0.45859219+0.00000000j ZIIIIIIIII
0.45859219+0.00000000j IZIIIIIIII
0.19735802+0.00000000j ZZIIIIIIII
0.10296096+0.00000000j YZZZYIIIII
-0.03439230+0.00000000j YIZZYIIIII
0.10296096+0.00000000j XZZZXIIIII
-0.03439230+0.00000000j XIZZXIIIII
-0.03243908+0.00000000j YZZZZZZZYI
0.00596838+0.00000000j YIZZZZZZYI
-0.03243908+0.00000000j XZZZZZZZXI
0.00596838+0.00000000j XIZZZZZZXI
0.03102012+0.00000000j YXXYIIIIII
-0.03102012+0.00000000j YYXXIIIIII
-0.03102012+0.00000000j XXYYIIIIII
0.03102012+0.00000000j XYYXIIIIII
-0.01878109+0.00000000j YXIXZZYIII
-0.01878109+0.00000000j YYIYZZYIII
-0.01878109+0.00000000j XXIXZZXIII
-0.01878109+0.00000000j XYIYZZXIII
0.00285607+0.00000000j XXZXIZIZIZ
0.00285607+0.00000000j XYZYIZIZIZ
0.00285607+0.00000000j XXIXZZZZZZ
0.00285607+0.00000000j XYIYZZZZZZ
0.10296096+0.00000000j IYZZZYIIII
-0.03439230+0.00000000j ZYZZZYIIII
0.1029609

## 5.3 compare

### 5.3.1 generators and paulixops

In [15]:
for idx, generator in enumerate(generators_by_qml):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops_by_qml[idx]}")

generator 1:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10], paulix_op: PauliX(wires=[10])
generator 2:   (1.0) [Z1 Z3 Z5 Z7 Z9 Z11], paulix_op: PauliX(wires=[11])


In [16]:
generators_by_tapering_ = get_generators_from_kernel(
    kernel_of_E(create_parity_check_matrix_E(*create_binary_matrix_G(Hami_ori)))
)
generators_by_tapering, paulix_ops_by_tapering = get_paulix_ops_from_generators(generators_by_tapering_)

for idx, generator in enumerate(generators_by_tapering.terms):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulix_ops_by_tapering.terms[idx]}")

generator 1: 1.00000000 ZIZIZIZIZIZI, paulix_op: 1.00000000 IIIIIIIIIIXI
generator 2: 1.00000000 IZIZIZIZIZIZ, paulix_op: 1.00000000 IIIIIIIIIIIX


### 5.3.2 optimal sectors

In [17]:
print(f'sectors by qml: {paulix_sector_by_qml}')
print(f'sectors by tapering: {optimal_sector(Hami_ori, generators_by_tapering, n_electrons)}')

sectors by qml: [-1, -1]
sectors by tapering: [-1, -1]
